In [1]:
!pip3 install altair
!pip3 install pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
import pandas as pd
import altair as alt
from IPython.display import display, HTML
import numpy as np
from sklearn.manifold import TSNE
display(HTML(data="""
<style> div#notebook-container { width: 95%; } div#menubar-container { width: 95%; } div#maintoolbar-container { width: 99%; } </style>

"""))

In [3]:
#initalize variables from various csv files
fns = ['prag_dates.csv', 'dates.csv', 'prag.csv', 'res.csv', 'ds_clumps.csv', 'ideology.csv', 'year.csv']

dfs = []
for i, df in enumerate(fns):
    dfs.append(pd.read_csv('./Newsbooks_Intermediary_Data/'+fns[i]))
prag_dates = dfs[0] 
dates = dfs[1]
prag = dfs[2] 
res =dfs[3]
ds_clumps =dfs[4] 
ideology =dfs[5]
year = dfs[6]

In [4]:
#all features
all_features = ['AcademicTerms',
       'AcademicWritingMoves', 'CharacterCognitiveAgency',
       'CharacterCognitiveStates', 'CharacterDialogueCues',
       'CharacterDisclosive', 'CharacterPersonNamesPronouns',
       'CharacterSubjective', 'CharacterTraitsTypes', 'CitationAuthority',
       'CitationControversy', 'CitationGeneral', 'CitationHedged',
       'CitationNegative', 'CitationNeutral', 'CitationSpeakerLookMood',
       'ConfidenceHedged', 'ConfidenceHigh', 'ConfidenceHighNegative',
       'ConfidenceLow', 'Contingent', 'Description', 'Facilitate',
       'FirstPerson', 'ForceStressed', 'Future', 'InformationChange',
       'InformationChangeNegative', 'InformationChangePositive',
       'InformationExposition', 'InformationPlace', 'InformationReportVerbs',
       'InformationStates', 'InformationTopics', 'Inquiry', 'Interactive',
       'MetadiscourseCohesive', 'MetadiscourseInteractive', 'NarrativeActs',
       'NarrativeBiographicalTime', 'NarrativeDuration',
       'NarrativeExpectation', 'NarrativeImmediacy',
       'NarrativePast', 'NarrativeSceneShifts', 'NarrativeSequence',
       'NarrativeSimultaneity', 'NarrativeTimeReference',
       'NarrativeTimeShifts', 'NarrativeTrends', 'NarrativeTurningPoint',
       'NarrativeVerbs', 'Negative', 'Orphaned', 'Positive', 'PublicTerms',
       'Reasoning', 'Responsibility', 'Strategic', 'SyntacticComplexity',
       'Uncertainty', 'Updates']

In [5]:
# 5-fold CV features
cv_features = ['!UNRECOGNIZED', '!UNTAGGED', 'CharacterDisclosive',
       'CitationAuthority', 'FirstPerson', 'InformationChange',
       'InformationChangePositive', 'InformationPlace',
       'InformationReportVerbs', 'Interactive', 'MetadiscourseCohesive',
       'NarrativeActs', 'NarrativeExpectation', 'NarrativePast',
       'NarrativeSequence', 'NarrativeSimultaneity', 'NarrativeTimeReference',
       'NarrativeTurningPoint', 'NarrativeVerbs', 'Negative', 'Orphaned',
       'PublicTerms', 'Reasoning', 'SyntacticComplexity', '<# Word Tokens>',
       '<# Punctuation Tokens>', '<# Tokens>']

# selected-features (unnormalized)
selected_unnormalized_features = ['!UNRECOGNIZED', 'AcademicTerms', 'CharacterCognitiveAgency',
       'CharacterCognitiveStates', 'CharacterDisclosive',
       'CitationControversy', 'ConfidenceHedged', 'ConfidenceHighNegative',
       'ConfidenceLow', 'Contingent', 'Description', 'FirstPerson',
       'InformationChangeNegative', 'InformationChangePositive',
       'InformationPlace', 'InformationReportVerbs', 'InformationStates',
       'Interactive', 'MetadiscourseInteractive', 'NarrativeActs',
       'NarrativeBiographicalTime', 'NarrativeExpectation',
       'NarrativeImmediacy', 'NarrativePast', 'NarrativeSceneShifts',
       'NarrativeSequence', 'NarrativeTimeReference', 'NarrativeTrends',
       'NarrativeTurningPoint', 'Negative', 'Orphaned', 'PublicTerms',
       'Reasoning', 'SyntacticComplexity', 'Updates', '<# Word Tokens>',
       '<# Punctuation Tokens>', '<# Tokens>', 'VID']

# selected features (normalized)
selected_normalized_features = ['!UNRECOGNIZED', 'CharacterCognitiveStates', 
                      'CharacterDialogueCues', 'CharacterPersonNamesPronouns',
                      'ConfidenceHedged', 'Description', 'FirstPerson', 'NarrativeSceneShifts',
                      'NarrativeTimeReference', 'Negative', 'Reasoning', '<# Punctuation Tokens>']

In [6]:
# change this line to select which features
res_features = res[selected_normalized_features]

In [7]:
# clean up res by filling in NA values with 0
res_clean = res_features.fillna(0)
# check if res_clean contains any NA values
res_clean[res_clean.isna().any(axis = 1)]

,!UNRECOGNIZED,CharacterCognitiveStates,CharacterDialogueCues,CharacterPersonNamesPronouns,ConfidenceHedged,Description,FirstPerson,NarrativeSceneShifts,NarrativeTimeReference,Negative,Reasoning,<# Punctuation Tokens>


In [8]:
# scaled csv from feature selection
df_scaled = pd.read_csv("df_scaled.csv")
df_scaled = df_scaled.iloc[:, 1:]

# normalized data + scaled
df_normalized_scaled = pd.read_csv("new_df_scaled_all.csv")
df_normalized_scaled = df_normalized_scaled.iloc[:, 1:]

In [9]:
# Calculating TSNE

# choose from df_scaled or df_normalized_scaled
# choose which set of features
X = df_scaled[selected_normalized_features]
X_embedded = TSNE(n_components=2, metric='euclidean', random_state = 36).fit_transform(X)

TSNE_df = pd.DataFrame(X_embedded, columns = ['TSNE_x', 'TSNE_y'])

/usr/local/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [10]:
# temporary copy of res to update TSNE values
new_res = res
new_res['TSNE_x'] = TSNE_df['TSNE_x']
new_res['TSNE_y'] = TSNE_df['TSNE_y']

In [11]:
# chart 1 plots TSNE_x vs TSNE_y
chart1 = alt.Chart(new_res).mark_circle(size=200).encode(
    x='TSNE_x',
    y='TSNE_y',
    color = alt.Color('Ideology', scale = alt.Scale(scheme = 'pastel2'))
).properties(
    width=500,
    height=500).interactive()

# chart 2 represents the three newsbooks (361, 413, 308) that are quite similar from Jiccard

data = pd.DataFrame({
    'TSNE_x': new_res.loc[[361, 413, 308],:]['TSNE_x'],
    'TSNE_y': new_res.loc[[361, 413, 308],:]['TSNE_y'],
    'color' : ['black', 'brown', 'hotpink']
})
chart2 = alt.Chart(data).mark_point(filled=True, size=200).encode(
    x='TSNE_x',
    y='TSNE_y',
    color=alt.Color('color', scale = None)
)
# print(res.loc[[361, 413, 308],:])
chart1 #+chart2

alt.Chart(...)